In [ ]:
# Use shared utils for persistent data dir and augmentation samples
import sys, os
from pathlib import Path
try:
    from shared import utils as u
except ImportError:
    repo_url = "https://github.com/anand-indx/dp-t25.git"; dest = "/content/dp-t25"
    if 'google.colab' in sys.modules and not os.path.exists(dest):
        import subprocess
        subprocess.run(['git', 'clone', '--depth', '1', repo_url, dest], check=False)
        sys.path.insert(0, dest)
    else:
        sys.path.insert(0, str(Path.cwd().parents[1]))
    from shared import utils as u

DATA_DIR = u.get_data_dir()
u.ensure_image_processing_samples(DATA_DIR)
print(f"📁 Augmentation samples located at: {DATA_DIR}")

## Basic Image Augmentation

Data augmentation is a technique to increase the diversity of your training set by applying random transformations to your images. This helps prevent overfitting.

**Our Goals:**
1.  Apply random horizontal flips.
2.  Apply random rotations.
3.  Combine multiple augmentations.

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
import random
from pathlib import Path

# Load our sample tile (create if missing)
# Use shared DATA_DIR from the setup cell
try:
    DATA_DIR
except NameError:
    from shared import utils as u
    DATA_DIR = u.get_data_dir()
    u.ensure_image_processing_samples(DATA_DIR)

(DATA_DIR).mkdir(parents=True, exist_ok=True)

tile_path = DATA_DIR / 'sample_tile.png'
if not tile_path.exists():
    # Create a simple tile from one of the generated samples
    src = DATA_DIR / 'histology_normal.jpg'
    if src.exists():
        img = Image.open(src).convert('RGB').resize((256, 256))
    else:
        img = Image.new('RGB', (256, 256), color='red')
    img.save(tile_path)

original_image = Image.open(tile_path)

### 1. Horizontal Flip

A horizontal flip is a common and effective augmentation.

In [ ]:
flipped_image = original_image.transpose(Image.FLIP_LEFT_RIGHT)

fig, axes = plt.subplots(1, 2, figsize=(8, 4))
axes[0].imshow(original_image)
axes[0].set_title('Original')
axes[1].imshow(flipped_image)
axes[1].set_title('Flipped')
plt.show()

### 2. Rotation

We can also rotate the image by a random angle.

In [ ]:
rotated_image = original_image.rotate(45)

fig, axes = plt.subplots(1, 2, figsize=(8, 4))
axes[0].imshow(original_image)
axes[0].set_title('Original')
axes[1].imshow(rotated_image)
axes[1].set_title('Rotated 45 degrees')
plt.show()

## ✅ Final Check

Let's create a simple augmentation function and test it.

In [ ]:
def augment_image(image):
    """Applies a random flip and rotation."""
    if random.random() > 0.5:
        image = image.transpose(Image.FLIP_LEFT_RIGHT)
    
    angle = random.choice([0, 90, 180, 270])
    image = image.rotate(angle)
    return image

augmented_image = augment_image(original_image)

# This check is visual, as the output is random.
# In a real test, you might check that the output image
# has the same size as the input.
assert original_image.size == augmented_image.size

print("SUCCESS: Augmentation function created and tested.")
plt.imshow(augmented_image)
plt.title('Randomly Augmented')
plt.show()